# Progetto MCS

Per la gestione della struttura dati e le operazioni elementari fra matrici è richiesto di partire da una libreria open-source, come Eigen, Armadillo, blas/lapack. Oppure, qualora il linguaggio di programmazione lo permetta utilizzare vettori e matrici già implementate al suo interno.


## Data Import

Impotazione delle librerie necessarie

In [2]:
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix, tril
from scipy.sparse.linalg import spsolve_triangular

Recupero dei dati

In [3]:
data = {
    "spa1" : 0,
    "spa2" : 0,
    "vem1" : 0,
    "vem2" : 0}
for x in data:
    data[x] = {
        "A" : mmread("data/" + x + ".mtx").tocsr(),
        "x" : 0,
        "b" : 0,
    }
    data[x]["x"] = np.array([1.0]*data[x]["A"].get_shape()[0])
    data[x]["b"] = np.array(data[x]["A"].dot(data[x]["x"]))
resTot = {}

Tolleranze e metodi

In [4]:
## Tolleranze
tols = [10**(-4), 10**(-6), 10**(-8), 10**(-10)]

## Metodi
metods = ["Jacoby", "Gauss-Seidel", "Gradiente", "Gradiente Coniugato"]

Dizionario contente le classi risolutrici

In [ ]:
import jacoby_mcs as ja
import gauss_seidel as gs
import gradiente as gr
import gradiente_coniugato as grc

solver = {}
solver["Jacoby"] = ja
solver["Gauss-Seidel"] = gs
solver["Gradiente"] = gr
solver["Gradiente Coniugato"] = grc

Condizione di arresto: $\frac{||Ax^{(k)}-b||}{||b||}<tol$

Reminder: 

1.   **Iniziare le iterazioni con il vettore x nullo**
2.   **tol = [$10^{-4}, 10^{-6}, 10^{-8}, 10^{-10}]$**
3.   **Dichiarare di non essere giunti a convergenza se k > maxiter dove maxiter lo scegliamo (>= 20000)**




## Metodo di Jacobi

In [ ]:
import jacoby_mcs as ja
metod = "Jacoby"
resTot = {}
resTot[metod] = {}
for el in data:
    resTot[metod][el] = []
    for tol in tols:
        res = ja.solve(data[el]["A"], data[el]["b"], data[el]["x"], tol)
        resTot[metod][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })

resTot

## Metodo di Gauß-Seidel

In [ ]:
import gauss_seidel as gs
metod = "Gauss-Seidel"
resTot[metod] = {}
for el in data:
    resTot[metod][el] = []
    for tol in tols:
        res = gs.solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, vectX=data[el]["x"])
        resTot[metod][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })

resTot

## Metodo del Gradiente


1. $r^{(k)} = b -Ax^{(k)}$
2. $y^{(k)} = Ar^{(k)}$
3. $a = (r^{(k)})^tr^{(k)}$
4. $b = (r^{(k)})^ty^{(k)}$
5. $\alpha_k = a/b$
6. $x^{(k+1)} = r^{(k)} \alpha_kr^{(k)}$

In [ ]:
import gradiente as gr

metod = "Gradiente"
solver = {}
solver[metod] = gr
resTot[metod] = {}
for el in data:
    resTot[metod][el] = []
    for tol in tols:
        res = solver[metod].solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, vectX=data[el]["x"])
        resTot[metod][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })
resTot

## Metodo del Gradiente coniugato

- Un vettore ottimale rispetto a una direzione d se d*r(k)=0
- x(k+1) è ottimale rispetto a r(k+1)
- x(k+1) = x(k) + a(k)d(k)
- a(k) = ( d(k)^t * r(k) ) / ( d(k)^t * Ad(k) )
- d(k+1) = r(k+1) - b(k)*d(k)
- b(k) = ( d(k)^t * Ar(k+1) ) / ( d(k)^t * Ad(k) )



In [ ]:
import gradiente_coniugato as grc
metod = "Gradiente Coniugato"
resTot = {}
resTot[metod] = {}
for el in data:
    resTot[metod][el] = []
    for tol in tols:
        res = grc.solve(data[el]["A"], data[el]["b"], data[el]["x"], tol)
        resTot[metod][el].append({
            "tol" : tol,
            "nIter" : res["nIter"],
            "time" : res["time"],
            "eRel" : res["eRel"] 
        })

resTot

## Risultati totali

In [ ]:
resTot = {}
for metod in solver:    
    resTot[metod] = {}
    for el in data:
        resTot[metod][el] = []
        for tol in tols:
            res = solver[metod].solve(mtxA=data[el]["A"], vectB=data[el]["b"], tol=tol, vectX=data[el]["x"])
            resTot[metod][el].append({
                "tol" : tol,
                "nIter" : res["nIter"],
                "time" : res["time"],
                "eRel" : res["eRel"] 
            })
resTot